In [13]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
import pandas as pd
import re
from nltk.corpus import stopwords
import string

stop_words = stopwords.words('english')
punctuations = list(string.punctuation)
stop_words.extend(punctuations)
real = pd.read_csv("True.csv")
fake = pd.read_csv("Fake.csv")

real['class'] = 1
fake['class'] = 0

df = pd.concat([real,fake])
df.reset_index(inplace = True)
df.drop('index', axis = 1, inplace = True)

def clean_text(text):
    index = text.name
    text = list(text)[0]
    if text == ' ':
        text = fill_title_in_text(index)
    text = re.sub(r'bit\S+', '', text) #removing bit.ly/*

    text = re.sub(r'\([^]]*\)', '', text) # removing parenthesis
    text = re.sub(r'\[[^]]*\]', '', text) # removing third brackets
    text = re.sub(r'([\.\\\+\*\?\[\^\]\$\(\)\{\}\!\<\>\|\:\-\,\"\”\“\‘])', '', text) #removing special characteres from the the words
    text = remove_stopwords(text)       
    return " ".join(text)

def remove_stopwords(text):
    new_text = []
    for i in text.lower().split():
        word = i.strip()
        if word not in stop_words:
            new_text.append(i)
    return new_text

def fill_title_in_text(index):
    text = df['title'].iloc[index]
    return text

df['text'] = (df['title'] + df['text']).to_frame(0).apply(clean_text, axis = 1)


In [16]:
df = df.drop(columns=["title", "subject", "date"])
df

,text,class
0,us budget fight looms republicans flip fiscal ...,1
1,us military accept transgender recruits monday...,1
2,senior us republican senator 'let mr mueller j...,1
3,fbi russia probe helped australian diplomat ti...,1
4,trump wants postal service charge 'much more' ...,1
...,...,...
44893,mcpain john mccain furious iran treated us sai...,0
44894,justice yahoo settles email privacy classactio...,0
44895,sunnistan us allied safe zone’ plan take terri...,0
44896,blow 700 million al jazeera america finally ca...,0


In [17]:
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained("bert-base-uncased")

text1 = "Replace me by any text you'd like."
text2 = "Remove the text"

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [18]:
text = df.text.values
text.shape

(44898,)

In [19]:
input_ids = [tokenizer.encode(i,
                              max_length=400,
                              pad_to_max_length=True, 
                              return_tensors="tf") for i in text]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [20]:
import numpy as np
output = [np.array(model(input_ids[i])['last_hidden_state']) for i in range(2000)]
output = np.asarray(output)
output.shape

(2000, 1, 400, 768)

In [22]:
output.shape

(2000, 1, 400, 768)

In [23]:
output = output.reshape(2000, 400, 768)

In [24]:
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
import tensorflow as tf 


fmodel = tf.keras.models.Sequential()
fmodel.add(LSTM(units = 50, activation = 'relu', recurrent_activation = 'tanh', return_sequences = True, recurrent_dropout = 0, dropout = 0.25 ))
fmodel.add(Dense(1, activation = 'sigmoid'))
fmodel.compile(optimizer = keras.optimizers.Adam(learning_rate = 0.01), loss = tf.keras.losses.binary_crossentropy, metrics = ['accuracy'])

In [25]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(output, df['class'][:2000], random_state = 0 )

In [26]:
history = fmodel.fit(x_train, y_train, validation_split = 0.1 , epochs = 15)
#Keyboard interrupt, as it was clear that accuracy would be 0%

Epoch 1/15
43/43 [==============================] - 36s 744ms/step - loss: nan - accuracy: 0.0193 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/15
43/43 [==============================] - 33s 780ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/15
43/43 [==============================] - 32s 747ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/15
43/43 [==============================] - 31s 729ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 5/15
30/43 [===================>..........] - ETA: 9s - loss: nan - accuracy: 0.0000e+00

KeyboardInterrupt: ignored